# konlpy 설치

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 53.2 MB/s 


# 데이터 수집

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs


url='http://api.koreafilm.or.kr/openapi-data2/wisenut/search_api/search_json2.jsp?collection=kmdb_new2&ServiceKey=735H810S461EXCLRX81S&'
url+='&detail=N&listCount=1000'
start1='20120101'
end1='20220315'
u1=url+'&releaseDts='+start1+'&releaseDte='+end1
start2='20010101'
end2='20120101'
u2=url+'&releaseDts='+start2+'&releaseDte='+end2
start3='19800101'
end3='20010101'
u3=url+'&releaseDts='+start3+'&releaseDte='+end3
req1 = requests.get(u1).text
req2 = requests.get(u2).text
req3 = requests.get(u3).text

# 데이터 전처리

In [3]:
import json
d1 = json.loads(req1)
d2 = json.loads(req2)
d3= json.loads(req3)
# con=d['Data'][0]['Result'][0]['plots']['plot'][0]['plotText']
# gen=d['Data'][0]['Result'][0]['genre']
xdata1=[d1['Data'][0]['Result'][i]['plots']['plot'][0]['plotText'] for i in range(len(d1['Data'][0]['Result']))]
ydata1=[d1['Data'][0]['Result'][i]['genre'] for i in range(len(d1['Data'][0]['Result']))]
xdata2=[d2['Data'][0]['Result'][i]['plots']['plot'][0]['plotText'] for i in range(len(d2['Data'][0]['Result']))]
ydata2=[d2['Data'][0]['Result'][i]['genre'] for i in range(len(d2['Data'][0]['Result']))]
xdata3=[d3['Data'][0]['Result'][i]['plots']['plot'][0]['plotText'] for i in range(len(d3['Data'][0]['Result']))]
ydata3=[d3['Data'][0]['Result'][i]['genre'] for i in range(len(d3['Data'][0]['Result']))]

In [4]:
xdata=xdata1+xdata2+xdata3
ydata=ydata1+ydata2+ydata3
df=pd.DataFrame(ydata,xdata)
df=df.reset_index()
df.columns=['줄거리','장르']
df['장르']

0                  액션,코메디,무협
1                  멜로드라마,동성애
2                     드라마,전쟁
3                         공포
4       SF,스릴러,어드벤처,미스터리,드라마
                ...         
1879                   멜로드라마
1880      멜로드라마,청춘영화,사회물(경향)
1881                멜로드라마,에로
1882                   멜로드라마
1883                 코메디,드라마
Name: 장르, Length: 1884, dtype: object

In [5]:
gen=[]
for i in df['장르'].tolist():
  gen.append(i.split(',')[0])
df['장르']=gen

In [6]:
df.drop(df[df['장르']==''].index, inplace=True)

In [7]:
df.loc[df['장르']=='멜로드라마','장르']='멜로/로맨스'
df.loc[df['장르']=='공포(호러)','장르']='공포'
df.장르.unique()

array(['액션', '멜로/로맨스', '드라마', '공포', 'SF', '코메디', '스릴러', '미스터리', '뮤직',
       '인물', '어드벤처', '사회', '아동', '무협', '지역', '범죄', '서부', '판타지', '가족',
       '전쟁', '기업ㆍ기관ㆍ단체', '자연ㆍ환경', '하이틴(고교)', '뮤지컬', '역사', '옴니버스',
       '시대극/사극', '인권', '에로', '갱스터', '청춘영화', '스포츠', '문화', '종교', '로드무비',
       '과학'], dtype=object)

In [8]:
import re
df['줄거리'] = df['줄거리'].apply( lambda x : re.sub("^[ㄱ-ㅎ가-힣0-9]*$", " ", x) )
df['줄거리'] = df['줄거리'].apply( lambda x : re.sub(r"\d+", " ", x) )
x_data=df['줄거리']
y_data=df['장르']

In [9]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels=encoder.fit_transform(y_data)
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35])

In [10]:
len(labels)

1869

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.3,random_state=121)

In [22]:
len(X_train)

1308

# 분석

In [12]:
from konlpy.tag import Okt

twitter = Okt()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=twitter.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(X_train)
# tfidf_matrix_train = tfidf_vect.transform(X_train)

# # Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
# tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
# tfidf_vect.fit(X_train)
# tfidf_matrix_train = tfidf_vect.transform(X_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7fc374aa04d0>>)

In [24]:
len(tfidf_vect.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


14377

In [14]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
rf_clf = RandomForestClassifier()
dt_clf = DecisionTreeClassifier()
knn_clf = KNeighborsClassifier(n_neighbors=8)

rf_clf.fit(tfidf_matrix_train,y_train)
dt_clf.fit(tfidf_matrix_train,y_train)
knn_clf.fit(tfidf_matrix_train,y_train)

KNeighborsClassifier(n_neighbors=8)

In [15]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(X_test)
rf_pred=rf_clf.predict(tfidf_matrix_test)
dt_pred=dt_clf.predict(tfidf_matrix_test)
knn_pred=knn_clf.predict(tfidf_matrix_test)

print('RandomForestClassifier  정확도: ',accuracy_score(y_test,rf_pred))
print('DecisionTreeClassifier  정확도: ',accuracy_score(y_test,dt_pred))
print('KNeighborsClassifier  정확도: ',accuracy_score(y_test,knn_pred))

RandomForestClassifier  정확도:  0.41711229946524064
DecisionTreeClassifier  정확도:  0.27807486631016043
KNeighborsClassifier  정확도:  0.31907308377896615


In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(tfidf_matrix_train,y_train)
gb_pred = gb_clf.predict(tfidf_matrix_test)
print('GradientBoostingClassifier  정확도: ',accuracy_score(y_test,gb_pred))

GradientBoostingClassifier  정확도:  0.39750445632798576


In [17]:
from sklearn import svm
clf = svm.SVC()
clf.fit(tfidf_matrix_train,y_train)
y_pred=clf.predict(tfidf_matrix_test)

print('svm  정확도: ',accuracy_score(y_test,y_pred))

svm  정확도:  0.4028520499108734


In [29]:
tfidf_matrix_train.shape

(1308, 14377)

In [18]:
from sklearn.model_selection import GridSearchCV

params = {
    'gamma':[0.1, 0.01, 0.001, 0.0001],
    'C' : [0.1, 0.01, 1, 2, 3], 
}
grid_cv = GridSearchCV(clf , param_grid=params , cv=2, refit=True )
grid_cv.fit(tfidf_matrix_test , y_test)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
pred = grid_cv.predict(tfidf_matrix_test)
print('정확도: ',accuracy_score(y_test,pred))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


최적 하이퍼 파라미터:
 {'C': 3, 'gamma': 0.1}
최고 예측 정확도: 0.3315
정확도:  0.5650623885918004


In [19]:
import joblib

#모델 파일로 저장
joblib.dump(grid_cv, 'genre.pkl')
joblib.dump(tfidf_vect, 'token.pkl')

joblib.dump(encoder, 'encoder.pkl')
#저장한 모델 로드
model = joblib.load('genre.pkl')


PicklingError: ignored

# 테스트중

In [25]:
a='asdf'
b=pd.Series(a)
b

0    asdf
dtype: object

In [93]:
import joblib
import numpy as np
import re
from konlpy.tag import Okt
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
def data():
    url = 'http://api.koreafilm.or.kr/openapi-data2/wisenut/search_api/search_json2.jsp?collection=kmdb_new2&ServiceKey=735H810S461EXCLRX81S&'
    url += '&detail=N&listCount=1000'
    start1 = '20120101'
    end1 = '20220315'
    u1 = url + '&releaseDts=' + start1 + '&releaseDte=' + end1
    start2 = '20010101'
    end2 = '20120101'
    u2 = url + '&releaseDts=' + start2 + '&releaseDte=' + end2
    start3 = '19800101'
    end3 = '20010101'
    u3 = url + '&releaseDts=' + start3 + '&releaseDte=' + end3
    req1 = requests.get(u1).text
    req2 = requests.get(u2).text
    req3 = requests.get(u3).text
    d1 = json.loads(req1)
    d2 = json.loads(req2)
    d3 = json.loads(req3)
    # con=d['Data'][0]['Result'][0]['plots']['plot'][0]['plotText']
    # gen=d['Data'][0]['Result'][0]['genre']
    xdata1 = [d1['Data'][0]['Result'][i]['plots']['plot'][0]['plotText'] for i in
              range(len(d1['Data'][0]['Result']))]
    ydata1 = [d1['Data'][0]['Result'][i]['genre'] for i in range(len(d1['Data'][0]['Result']))]
    xdata2 = [d2['Data'][0]['Result'][i]['plots']['plot'][0]['plotText'] for i in
              range(len(d2['Data'][0]['Result']))]
    ydata2 = [d2['Data'][0]['Result'][i]['genre'] for i in range(len(d2['Data'][0]['Result']))]
    xdata3 = [d3['Data'][0]['Result'][i]['plots']['plot'][0]['plotText'] for i in
              range(len(d3['Data'][0]['Result']))]
    ydata3 = [d3['Data'][0]['Result'][i]['genre'] for i in range(len(d3['Data'][0]['Result']))]
    xdata = xdata1 + xdata2 + xdata3
    ydata = ydata1 + ydata2 + ydata3
    df = pd.DataFrame(ydata, xdata)
    df = df.reset_index()
    df.columns = ['줄거리', '장르']
    gen = []
    for i in df['장르'].tolist():
        gen.append(i.split(',')[0])
    df['장르'] = gen
    df.drop(df[df['장르'] == ''].index, inplace=True)
    df.loc[df['장르'] == '멜로드라마', '장르'] = '멜로/로맨스'
    df.loc[df['장르'] == '공포(호러)', '장르'] = '공포'
    df['줄거리'] = df['줄거리'].apply(lambda x: re.sub("^[ㄱ-ㅎ가-힣0-9]*$", " ", x))
    df['줄거리'] = df['줄거리'].apply(lambda x: re.sub(r"\d+", " ", x))
    x_data = df['줄거리']
    y_data = df['장르']
    encoder = LabelEncoder()
    labels = encoder.fit_transform(y_data)
    X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.3, random_state=121)
    return X_train

class MovieService:
    

    def tw_tokenizer(text):
        twitter = Okt()
        tokens_ko = twitter.morphs(text)
        return tokens_ko

    def genre_test(self, da:str):
        model = joblib.load('genre.pkl')
        tfidf_vect = TfidfVectorizer(tokenizer=MovieService.tw_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
        df=da
        tfidf_vect.fit(df)
        decoder = joblib.load('encoder.pkl')
        input = da
        input = input.apply(lambda x: re.sub("[^?a-zA-Z]", " ", x))
        input = input.apply(lambda x: re.sub(r"\d+", " ", x))
        tfidf_matrix_test = tfidf_vect.transform(input)
        pred = model.predict(tfidf_matrix_test)
        genre = decoder.inverse_transform(pred)
        return genre


In [94]:
asdf='사랑했던 연인을 다시 만나 행복한 나날을 보내는 이야기'
a=pd.DataFrame(asdf,columns=['줄거리'],index=['0'])
a['줄거리']

0    사랑했던 연인을 다시 만나 행복한 나날을 보내는 이야기
Name: 줄거리, dtype: object

In [95]:

service=MovieService()
levels = service.genre_test(a['줄거리'])
print(levels)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


ValueError: ignored

In [99]:
input = asdf
print(input)
input = re.sub("^[ㄱ-ㅎ가-힣0-9]*$", " ", input)
print('a',input)
# input = re.sub(r"\d+", " ", input)
# input

사랑했던 연인을 다시 만나 행복한 나날을 보내는 이야기
a 사랑했던 연인을 다시 만나 행복한 나날을 보내는 이야기
